# Create Feature_Usage Objects in ADLSTE_COA 

In [1]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


<br/><br/><br/>
## COA Populated Tables - Feature_Usage_Log

In [ ]:
drop table adlste_coa_stg.stg_dat_feature_usage_log

In [ ]:
-- GTT table
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_feature_usage_log 
(
  Site_ID             VARCHAR(128) CHARACTER SET LATIN NOT NULL
, LogDate             DATE FORMAT 'yyyy-mm-dd' NOT NULL
, BitPOS              INTEGER NOT NULL
, Feature_Name        VARCHAR(256) CHARACTER SET LATIN NOT NULL
, User_Bucket         VARCHAR(128) CHARACTER SET LATIN 
, User_Department     VARCHAR(128) CHARACTER SET LATIN
, User_Subdepartment  VARCHAR(128) CHARACTER SET LATIN
, User_Region         VARCHAR(128) CHARACTER SET LATIN
, Feature_Usage_Cnt   INTEGER
) NO PRIMARY INDEX 
ON COMMIT PRESERVE ROWS;

In [72]:
rename table adlste_coa.coat_dat_feature_usage_log to adlste_coa.coat_dat_feature_usage_log_tmp

Success: 2 rows affected

In [73]:
-- PERM table
CREATE MULTISET TABLE adlste_coa.coat_dat_feature_usage_log 
(
  Site_ID             VARCHAR(128) CHARACTER SET LATIN  NOT NULL
, LogDate             DATE FORMAT 'yyyy-mm-dd' NOT NULL
, Feature_ID          INTEGER NOT NULL
, Feature_Name        VARCHAR(256) CHARACTER SET LATIN NOT NULL
, User_Bucket         VARCHAR(128) CHARACTER SET LATIN 
, User_Department     VARCHAR(128) CHARACTER SET LATIN
, User_Subdepartment  VARCHAR(128) CHARACTER SET LATIN
, User_Region         VARCHAR(128) CHARACTER SET LATIN
, Feature_Usage_Cnt   INTEGER
, Run_Id              INTEGER
) PRIMARY INDEX ( Site_Id ,LogDate ,Feature_Id ) 
PARTITION BY RANGE_N(LogDate BETWEEN DATE '2019-01-01' AND DATE '2021-12-31' EACH INTERVAL '1' DAY 
                     ,NO RANGE OR UNKNOWN);

Success: 0 rows affected

In [79]:
insert into adlste_coa.coat_dat_feature_usage_log
select Site_ID, LogDate, f.feature_ID, f.Feature_Name, 
User_Bucket, User_Department, User_Subdepartment, User_Region, Feature_Usage_Cnt, l.Run_Id
from adlste_coa.coat_dat_feature_usage_log_tmp as l
join adlste_coa.coat_dim_feature as f
  on l.Feature_ID = f.Feature_ID

Success: 10 rows affected

In [80]:
drop table adlste_coa.coat_dat_feature_usage_log_tmp

Success: 38 rows affected

In [81]:

-- view
replace view adlste_coa.coa_dat_feature_usage_log as locking row for access
select * from adlste_coa.coat_dat_feature_usage_log

Success: 11 rows affected

In [112]:
REPLACE PROCEDURE adlste_coa.SP_dat_feature_usage_log ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_feature_usage_log';
    SET callingsp = 'adlste_coa.sp_dat_feature_usage_log';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(LogDate) as StartDate
    ,max(LogDate) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_feature_usage_log
    Group by 1
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START',  '') ;

    DELETE FROM adlste_coa.coat_dat_feature_usage_log WHERE (Site_ID, LogDate ) IN
    (Select Site_ID, LogDate From adlste_coa_stg.stg_dat_feature_usage_log );
    
    Insert into adlste_coa.coat_dat_feature_usage_log 
    SELECT
      l.Site_ID
    , l.LogDate
    , coalesce(f.Feature_ID, -1) as Feature_ID
    , coalesce(f.Feature_Name, trim(l.Feature_Name)||' ('|| trim(l.BitPos)||')') as Feature_Name
    , l.User_Bucket
    , l.User_Department
    , l.User_Subdepartment
    , l.User_Region
    , l.Feature_Usage_Cnt  
    , :runid
    FROM adlste_coa_stg.stg_dat_feature_usage_log as l
    LEFT OUTER JOIN adlste_coa.coat_dim_feature as f
      on (f.Feature_Name) = (l.Feature_Name);

    delete from adlste_coa_stg.stg_dat_feature_usage_log;
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

Success: 0 rows affected

### Test:

In [118]:
-- put records into STG
insert into adlste_coa_stg.stg_dat_feature_usage_log
SELECT l.Site_ID, l.LogDate, l.Feature_ID, f.Feature_Name, 
 l.User_Bucket, l.User_Department, l.User_Subdepartment, l.User_Region, 
 l.Feature_Usage_Cnt   
FROM adlste_coa.coa_dat_feature_usage_log as l
JOIN adlste_coa.coa_dim_feature as f
  on l.Feature_ID = f.Feature_ID
Qualify l.LogDate = min(LogDate) over()
and Site_ID = max(Site_ID)over()

Success: 2 rows affected

In [119]:
call adlste_coa.SP_dat_feature_usage_log('test')

Success: 0 rows affected

In [120]:
select top 10 * from adlste_coa.coat_dat_feature_usage_log

Site_ID,LogDate,Feature_ID,Feature_Name,User_Bucket,User_Department,User_Subdepartment,User_Region,Feature_Usage_Cnt,Run_Id
APPLE57CRESCENT,2020-05-08,18300,Partition Level Locking,OUTBOUND,SALESOPS,na,na,0,178
APPLE57CRESCENT,2020-05-08,19500,SET TRANSFORM,SYSTEM,OTHERS,na,na,0,178
APPLE57CRESCENT,2020-05-09,22100,Not System-Default Map,ADHOC,APPLECARE,na,na,0,178
APPLE57CRESCENT,2020-05-08,26600,Partial Redistribution Partial Duplication,SEMANTIC,SYSTEM,na,na,0,178
APPLE62SEDONA,2020-05-08,22600,SET Table,DASHBOARD,SYSTEM,na,na,0,2089
APPLE46REDWOOD,2020-05-08,26500,In-Memory Bulk Qualification Enhancements,DASHBOARD,SALESOPS,na,na,0,190
APPLE46REDWOOD,2020-05-08,18600,Array Data Type,SEMANTIC,ITUNES,na,na,0,190
APPLE46REDWOOD,2020-05-08,23500,Fallback,BOBJ,APPLECARE,na,na,0,190
APPLE57CRESCENT,2020-05-12,26000,Incremental Planning and Execution,ADHOC,OTHERS,na,na,0,178
APPLE62SEDONA,2020-05-08,22200,Primary Index,BOBJ,SALESOPS,na,na,0,2089


<br/><br/><br/>
## View:  adlste_coa.coa_dat_feature

In [ ]:
replace view adlste_coa.coa_dat_feature as locking row for access
SELECT 
 coalesce(Site_ID,'') as Site_ID
,coalesce(LogDate,cast('9999-12-31' as date)) as LogDate
,Product_Category
,Feature_Category
,Feature_SubCategory
,Feature_Name
,Feature_Usage_Cnt as Usage_Cnt
,DBQL_FeatureInfo_Flag as DBQL_Flg
,MBO_Ind
,Feature_Weight as MBO_Weight
,User_Bucket
,User_Department
,Objective_Name
,Solution_Type
,Business_Value_Desc
,User_Experience_Level_Name
,Initiator_Role_Category
,Differentiator_Ind 
,Enabled_Thru
,Feature_SQL_URL
,Playbook_URL
from adlste_coa.coa_dim_feature f
left outer join adlste_coa.coa_dat_feature_usage_log d
  on d.Feature_ID = f.Feature_ID 

In [ ]:
select * from adlste_coa.coa_dat_feature 
where Site_ID = 'TDCloud14TD03' and LogDate = '2020-06-10' 
order by Product_Category, Feature_Category, Feature_SubCategory, Feature_Name

<br/><br/><br/>
## Feature_Usage_LOG  sql for COA

In [ ]:
select  * from adlste_coa_stg.stg_dat_feature_usage_log